In [2]:
import requests
import psycopg2
import os
import json
from datetime import datetime, timedelta


def notify_api_run(api_key, base_url, message_id):
    headers = {
        'accept': 'application/json',
        'Authorization': f'ApiKey-v1 {api_key}'
    }
    url = base_url + message_id
    response = requests.request("GET", url, headers=headers)
    resp_str = ''
    if response.status_code == 200:
        if 'status' in response.json():
            if not response.json()['status'] == 'delivered':
                resp_str += '----------\n'
                resp_str += response.json()['status']
                resp_str += '\n'
                resp_str += json.dumps(response.json(), indent=2)
                resp_str += '\n'
                resp_str += '----------\n'
        else:
            resp_str += '----------\n'
            resp_str += json.dumps(response.json(), indent=2)
            resp_str += '\n'
            resp_str += '----------\n'
    else:
        resp_str += '----------\n'
        resp_str += response.status_code
        resp_str += '\n'
        resp_str += json.dumps(response.json(), indent=2)
        resp_str += '\n'
        resp_str += '----------\n'
    return resp_str


def check_notifications():
    conn = psycopg2.connect(user=os.getenv('DB_USER', 'postgres'),
                            host=os.getenv('DB_HOST', 'localhost'),
                            port=os.getenv('DB_PORT', ''),
                            dbname=os.getenv('DB_NAME', 'notify'))
    cur = conn.cursor()
    select_q = "SELECT gc_notify_response_id FROM public.notification_history where sent_date >= now() - interval '1 day' and provider_code='GC_NOTIFY' order by sent_date desc;"
    cur.execute(select_q)
    resp_ids = cur.fetchall()
    datestr = datetime.strftime(datetime.now() - timedelta(1), '%Y-%m-%d')
    filename = os.path.join(os.getcwd(), r'data/') + 'gc-notify-failures-' + datestr + '.csv'

    with open(filename, 'w') as f:
        for message_id in resp_ids:
            resp_str = notify_api_run(
                os.getenv('GC_NOTIFY_KEY', ''),
                os.getenv('GC_NOTIFY_URL', 'https://api.notification.canada.ca/v2/notifications/'),
                message_id[0])
            if len(resp_str) > 0:
                f.write(resp_str + '\n')
        f.write('\n\n')


if __name__ == '__main__':
    check_notifications()